<a href="https://colab.research.google.com/github/miwojtkowska/dw_matrix_car/blob/master/dw_matrix2_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/dw_matrix2/dw_matrix_car')

In [8]:
df = pd.read_hdf('/content/drive/My Drive/Colab Notebooks/dw_matrix2/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [10]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]

#usunięcie kolumn, które mają price w nazwie
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
  return round(np.mean(scores),1), round(np.std(scores),1)

## Decision tree

In [14]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015937)

## Random forest

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=2020)
run_model(model, cat_feats)

(-18774.078311945963, 56.07799419652943)

## XGBoost

In [18]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[12:27:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:28:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:28:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.4, 74.3)

In [19]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp = PermutationImportance(m, random_state = 2020).fit(X,y)
eli5.show_weights(imp, feature_names = cat_feats)

[12:29:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1190 ± 0.0036,param_napęd__cat
0.1130 ± 0.0029,param_rok-produkcji__cat
0.1084 ± 0.0026,param_stan__cat
0.0610 ± 0.0019,param_skrzynia-biegów__cat
0.0577 ± 0.0016,param_faktura-vat__cat
0.0483 ± 0.0015,param_moc__cat
0.0274 ± 0.0009,param_marka-pojazdu__cat
0.0238 ± 0.0012,feature_kamera-cofania__cat
0.0211 ± 0.0007,param_typ__cat
0.0175 ± 0.0009,seller_name__cat


In [21]:
cat_feats = ['param_napęd__cat', 'param_rok-produkcji__cat', 'param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'seller_name__cat', 'param_pojemność-skokowa__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[12:40:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:40:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:40:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.2, 65.4)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [23]:
cat_feats = ['param_napęd__cat', 'param_rok-produkcji', 'param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc__cat', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'seller_name__cat', 'param_pojemność-skokowa__cat', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[12:43:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:43:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:43:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.9, 27.9)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
#poejmnosc skokowa
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

[12:48:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.3, 62.5)

In [30]:
cat_feats = [
             'param_napęd__cat', 
             'param_rok-produkcji', 
             'param_stan__cat', 
             'param_skrzynia-biegów__cat', 
             'param_faktura-vat__cat', 
             'param_moc', 
             'param_marka-pojazdu__cat', 
             'feature_kamera-cofania__cat', 
             'param_typ__cat', 
             'seller_name__cat', 
             'param_pojemność-skokowa', 
             'feature_wspomaganie-kierownicy__cat', 
             'param_model-pojazdu__cat', 
             'param_wersja__cat', 
             'param_kod-silnika__cat', 
             'feature_system-start-stop__cat', 
             'feature_asystent-pasa-ruchu__cat', 
             'feature_czujniki-parkowania-przednie__cat', 
             'feature_łopatki-zmiany-biegów__cat', 
             'feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[12:52:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:52:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.2, 72.8)